In [26]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('../../data/df_fin2.csv')
df.shape

(806367, 32)

---

# 1. 단일 구매자 / 중복 구매자

## 1-1. 단일 구매자 vs 중복 구매자

### 1-1-1. 고객 수 기준 비율

In [13]:
# 고객별 구매 횟수
cust_cnt = (
    df
    .groupby('customer_id')
    .size()
    .reset_index(name='purchase_cnt')
)

# 구매자 유형 구분
cust_cnt['buyer_type'] = '중복 구매자'
cust_cnt.loc[cust_cnt['purchase_cnt'] == 1, 'buyer_type'] = '단일 구매자'

# 고객 수 기준 집계
buyer_cnt = (
    cust_cnt
    .groupby('buyer_type')
    .size()
    .reset_index(name='customer_cnt')
)

# 비율 계산
buyer_cnt['pct'] = (
    buyer_cnt['customer_cnt']
    / buyer_cnt['customer_cnt'].sum()
    * 100
).round(2)

print('** 단일 / 중복 구매자 **')
display(buyer_cnt)

** 단일 / 중복 구매자 **


,buyer_type,customer_cnt,pct
0,단일 구매자,176437,49.9
1,중복 구매자,177123,50.1


### 1-1-2. 거래 건수 기준 비율

In [14]:
# 거래 데이터에 buyer_type 붙이기
df_with_type = df.merge(
    cust_cnt[['customer_id', 'buyer_type']],
    on='customer_id',
    how='left'
)

# 거래 건수 기준 집계
buyer_tran_cnt = (
    df_with_type
    .groupby('buyer_type')
    .size()
    .reset_index(name='transaction_cnt')
)

# 비율 계산
buyer_tran_cnt['pct'] = (
    buyer_tran_cnt['transaction_cnt']
    / buyer_tran_cnt['transaction_cnt'].sum()
    * 100
).round(2)

display(buyer_tran_cnt)


,buyer_type,transaction_cnt,pct
0,단일 구매자,176437,21.88
1,중복 구매자,629930,78.12


## 1-2. 중복 구매 횟수

In [ ]:
repeat_buyers = cust_cnt[cust_cnt['purchase_cnt'] >= 2]

purchase_cnt_summary = (
    repeat_buyers['purchase_cnt']
    .value_counts()
    .reset_index()
)

purchase_cnt_summary.columns = ['purchase_cnt', 'customer_cnt']

# purchase_cnt_summary = purchase_cnt_summary.sort_values('purchase_cnt', ascending=False)
display(purchase_cnt_summary)

,purchase_cnt,customer_cnt
43,59,1
42,52,1
40,46,1
39,44,1
32,43,3
41,42,1
38,41,2
35,38,2
30,37,4
34,36,3


## 1-3. 중복 구매자의 채널 선호도

In [15]:
# 1) merged df에 buyer_type 붙이기
df_with_type = df.merge(
    cust_cnt[['customer_id', 'buyer_type']],
    on='customer_id',
    how='left'
)

# 2) buyer_type x channel 거래 건수
buyer_channel_cnt = (
    df_with_type
    .groupby(['buyer_type', 'channel'])
    .size()
    .reset_index(name='transaction_cnt')
)

# 3) buyer_type 내 채널 비율(%)
buyer_channel_cnt['pct'] = (
    buyer_channel_cnt['transaction_cnt']
    / buyer_channel_cnt.groupby('buyer_type')['transaction_cnt'].transform('sum')
    * 100
).round(2)

# 4) 보기 좋게 피벗(온라인/오프라인 비율 한눈에)
buyer_channel_ratio = (
    buyer_channel_cnt
    .pivot(index='buyer_type', columns='channel', values='pct')
    .reset_index()
)

display(buyer_channel_ratio)


channel,buyer_type,오프라인,온라인
0,단일 구매자,39.60,60.40
1,중복 구매자,28.09,71.91


In [16]:
# ===== 0) 중복 구매자 ID 구하기 =====
cust_purchase_cnt = (
    df
    .groupby('customer_id')
    .size()
    .reset_index(name='purchase_cnt')
)

repeat_ids = set(
    cust_purchase_cnt.loc[cust_purchase_cnt['purchase_cnt'] >= 2, 'customer_id']
)

df_repeat = df[df['customer_id'].isin(repeat_ids)].copy()


# ===== 1차 분석: 중복 구매자 × index_group_name (타겟군) =====
repeat_target_1st = (
    df_repeat
    .groupby('index_group_name')
    .size()
    .reset_index(name='transaction_cnt')
    .sort_values('transaction_cnt', ascending=False)
)

display(repeat_target_1st.head(20))


# ===== 2차 분석: 중복 구매자 × product_group_name (제품군) =====
repeat_prodgroup_2nd = (
    df_repeat
    .groupby('product_group_name')
    .size()
    .reset_index(name='transaction_cnt')
    .sort_values('transaction_cnt', ascending=False)
)

display(repeat_prodgroup_2nd.head(20))

,index_group_name,transaction_cnt
2,Ladieswear,408591
1,Divided,143126
3,Menswear,33508
4,Sport,22449
0,Baby/Children,22256


,product_group_name,transaction_cnt
6,Garment Upper body,249726
5,Garment Lower body,140266
4,Garment Full body,72511
12,Swimwear,53301
13,Underwear,47380
0,Accessories,30963
10,Shoes,15915
11,Socks & Tights,12032
9,Nightwear,6711
15,Unknown,884


In [ ]:
repeat_season_cnt = (
    df_repeat
    .groupby('product_season')
    .size()
    .reset_index(name='transaction_cnt')
    .sort_values('transaction_cnt', ascending=False)
)

repeat_season_cnt['pct'] = (
    repeat_season_cnt['transaction_cnt']
    / repeat_season_cnt['transaction_cnt'].sum()
    * 100
).round(2)

display(repeat_season_cnt)

,product_season,transaction_cnt,pct
0,All-Season,422447,67.06
2,SS,155137,24.63
1,FW,52346,8.31


In [ ]:
# # 전제: df_repeat = 반복 구매자(= purchase_cnt>=2)만 필터링된 df
# #       df_repeat에는 최소 customer_id, t_dat(구매일), product_season(FW/SS/All-Season) 컬럼이 있어야 함

# # 0) 구매일 정렬용 타입 보정 + 정렬
# df_repeat = df_repeat.copy()
# df_repeat['t_dat'] = pd.to_datetime(df_repeat['t_dat'])
# df_repeat = df_repeat.sort_values(['customer_id', 't_dat'])

# # 1) 고객별로 연속 구매의 "이전 시즌 -> 다음 시즌" 전이(transition) 만들기
# df_repeat['prev_season'] = df_repeat.groupby('customer_id')['product_season'].shift(1)
# df_repeat['next_season'] = df_repeat['product_season']

# # 2) 첫 구매(이전 시즌 없음) 제거
# transitions = df_repeat.dropna(subset=['prev_season']).copy()

# # 3) 전이 패턴 집계: FW->FW, FW->SS ... 거래건수
# season_transition_cnt = (
#     transitions
#     .groupby(['prev_season', 'next_season'])
#     .size()
#     .reset_index(name='transition_cnt')
#     .sort_values('transition_cnt', ascending=False)
# )

# # 4) 같은 시즌 반복(FW->FW, SS->SS, All-Season->All-Season) 여부 컬럼 추가
# season_transition_cnt['is_same_season'] = (
#     season_transition_cnt['prev_season'] == season_transition_cnt['next_season']
# )

# display(season_transition_cnt)

# # 5) 같은 시즌 반복 비율(%) 요약
# same_season_summary = (
#     season_transition_cnt
#     .groupby('is_same_season')['transition_cnt']
#     .sum()
#     .reset_index()
# )

# same_season_summary['pct'] = (
#     same_season_summary['transition_cnt']
#     / same_season_summary['transition_cnt'].sum()
#     * 100
# ).round(2)

# display(same_season_summary)

# # 6) 고객별로도 "같은 시즌 반복" 비율 계산(고객 단위)
# cust_same_ratio = (
#     transitions
#     .assign(is_same=lambda x: x['prev_season'] == x['next_season'])
#     .groupby('customer_id')['is_same']
#     .mean()
#     .reset_index(name='same_season_ratio')
# )

# cust_same_ratio['same_season_ratio'] = (cust_same_ratio['same_season_ratio'] * 100).round(2)
# display(cust_same_ratio.head())


In [21]:
df_time = df.copy()
df_time['t_dat'] = pd.to_datetime(df_time['t_dat'])

# 고객별 구매일 정렬 후 구매 순서 부여 (1=첫구매, 2=재구매)
df_time = df_time.sort_values(['customer_id', 't_dat'])
df_time['purchase_order'] = df_time.groupby('customer_id').cumcount() + 1

# 첫 구매일 / 두 번째 구매일 추출
first_purchase = (
    df_time[df_time['purchase_order'] == 1][['customer_id', 't_dat']]
    .rename(columns={'t_dat': 'first_purchase_date'})
)

second_purchase = (
    df_time[df_time['purchase_order'] == 2][['customer_id', 't_dat']]
    .rename(columns={'t_dat': 'second_purchase_date'})
)

# 재구매한 고객만 남김
repurchase_df = first_purchase.merge(second_purchase, on='customer_id', how='inner')

# 첫 구매 → 재구매까지 걸린 시간(일)
repurchase_df['days_to_repurchase'] = (
    repurchase_df['second_purchase_date'] - repurchase_df['first_purchase_date']
).dt.days

# 전체 평균/중앙값
print("전체 평균 재구매 간격(일):", round(repurchase_df['days_to_repurchase'].mean(), 2))
print("전체 중앙값 재구매 간격(일):", round(repurchase_df['days_to_repurchase'].median(), 2))

# 연령대 붙이기(고객별 1개만)
age_map = df_time[['customer_id', 'age_segment']].drop_duplicates('customer_id')
repurchase_df = repurchase_df.merge(age_map, on='customer_id', how='left')

# 연령대별 평균/중앙값/표본수
age_repurchase_gap = (
    repurchase_df
    .groupby('age_segment')['days_to_repurchase']
    .agg(avg_days='mean', median_days='median', customer_cnt='count')
    .reset_index()
    .sort_values('avg_days')
)

age_repurchase_gap[['avg_days', 'median_days']] = age_repurchase_gap[['avg_days', 'median_days']].round(2)

display(age_repurchase_gap)

전체 평균 재구매 간격(일): 63.59
전체 중앙값 재구매 간격(일): 37.0


,age_segment,avg_days,median_days,customer_cnt
2,30대,59.15,31.0,35574
0,10대,60.60,33.0,2889
3,40대,62.19,35.0,27310
4,50대,64.67,39.0,29640
1,20대,65.46,40.0,73708
5,60대 이상,67.99,41.0,8002


In [25]:
import pandas as pd

# 전제: df_merged = 3개 테이블 merge 완료된 df
# 필요 컬럼: customer_id, channel, index_group_name(또는 product_group_name), article_id(없어도 size로 카운트 가능)

df_all = df.copy()

# 1) 반복 구매자(>=2회) 식별
cust_purchase_cnt = (
    df_all
    .groupby('customer_id')
    .size()
    .reset_index(name='purchase_cnt')
)

repeat_buyers = cust_purchase_cnt[cust_purchase_cnt['purchase_cnt'] >= 2]

df_repeat = df_all.merge(
    repeat_buyers[['customer_id', 'purchase_cnt']],
    on='customer_id',
    how='inner'
)

# 2) 고객별 "주 채널"(가장 많이 이용한 채널) 구하기
main_channel_by_cust = (
    df_repeat
    .groupby(['customer_id', 'channel'])
    .size()
    .reset_index(name='cnt')
    .sort_values(['customer_id', 'cnt'], ascending=[True, False])
    .drop_duplicates('customer_id')
)

# 주 채널 분포(비율)
main_channel_summary = (
    main_channel_by_cust['channel']
    .value_counts(normalize=True)
    .mul(100)
    .round(2)
    .reset_index()
)

main_channel_summary.columns = ['main_channel', 'pct']  # <- 무조건 고정
display(main_channel_summary)

# 3) 고객별 "주요 상품군"(가장 많이 구매한 카테고리) 구하기
#    1순위: index_group_name / 없으면 product_group_name으로 바꿔서 쓰기
category_col = 'index_group_name'  # 필요시 'product_group_name'로 변경

main_category_by_cust = (
    df_repeat
    .groupby(['customer_id', category_col])
    .size()
    .reset_index(name='cnt')
    .sort_values(['customer_id', 'cnt'], ascending=[True, False])
    .drop_duplicates('customer_id')
)

main_category_summary = (
    main_category_by_cust[category_col]
    .value_counts(normalize=True)
    .mul(100)
    .round(2)
    .reset_index()
)

main_category_summary.columns = ['main_category', 'pct']  # <- 무조건 고정
display(main_category_summary)

# 4) 구매 빈도 요약
purchase_freq_summary = (
    repeat_buyers['purchase_cnt']
    .agg(avg_purchase_cnt='mean', median_purchase_cnt='median', max_purchase_cnt='max')
    .to_frame(name='value')
    .reset_index()
)

purchase_freq_summary['value'] = purchase_freq_summary['value'].round(2)
display(purchase_freq_summary)

# 5) 전략용 한 줄 요약 테이블
strategy_summary = pd.DataFrame({
    '항목': ['주 구매 채널', '주요 상품군', '평균 구매 횟수', '중앙값 구매 횟수', '최대 구매 횟수'],
    '요약': [
        main_channel_summary.iloc[0]['main_channel'],
        main_category_summary.iloc[0]['main_category'],
        round(repeat_buyers['purchase_cnt'].mean(), 2),
        int(repeat_buyers['purchase_cnt'].median()),
        int(repeat_buyers['purchase_cnt'].max())
    ]
})

display(strategy_summary)


,main_channel,pct
0,온라인,63.96
1,오프라인,36.04


,main_category,pct
0,Ladieswear,64.79
1,Divided,27.36
2,Baby/Children,4.56
3,Menswear,2.31
4,Sport,0.99


,index,value
0,avg_purchase_cnt,3.56
1,median_purchase_cnt,3.00
2,max_purchase_cnt,59.00


,항목,요약
0,주 구매 채널,온라인
1,주요 상품군,Ladieswear
2,평균 구매 횟수,3.56
3,중앙값 구매 횟수,3
4,최대 구매 횟수,59
